<center><font size =6, color="marquee"> Twitter US Airline Sentiment Classification using 
    navie bayes generative classifier </font></center>

  https://www.kaggle.com/crowdflower/twitter-airline-sentiment
  
  Data can be downloaded from the following link. You have to login to the kaggle site to get access to the data.
Unzip the data in folder where notebook is running. There should be a file called **Tweet.csv**

https://www.kaggle.com/crowdflower/twitter-airline-sentiment/downloads/twitter-airline-sentiment.zip/2

<img src="https://images.unsplash.com/photo-1529990131237-cfa5ce9517b6?ixlib=rb-1.2.1&auto=format&fit=crop&w=1949&q=80"  width="900" height="400">

**Problem statement:** classify sentiment based on text as positive or negative* using **naive bayes** in supervised machine setting.

In [16]:
%matplotlib inline 
import numpy as np
import pandas as pd # for data analysis
import matplotlib.pyplot as plt # for plotting



In [17]:
#! pip install scikit-learn
#! pip install gensim
#!pip install -U setuptools
#!pip install pattern

In [18]:
data_url = 'Tweets.csv' 

In [19]:
!head -n 10  Tweets.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


Clearly it is a comma separated csv file.

In [20]:
airlines_sentiment_df = pd.read_csv('Tweets.csv')
airlines_sentiment_df.head(6) # there are other functions like tail and sample to check record in dataset

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)


# Selecting observation with positive and negative sentiment only.

In [21]:
airlines_sentiment_df = airlines_sentiment_df[airlines_sentiment_df.airline_sentiment.isin(['positive', 'negative'])]

In [22]:
airlines_sentiment_df.sample(10)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
939,570000287080951808,negative,1.0000,Cancelled Flight,0.6566,United,NaN,MirandaAtx,NaN,0,@united Exhausted &amp; frustrated! Link to a ...,NaN,2015-02-23 15:20:33 -0800,"Houston, TX",Central Time (US & Canada)
14628,569588651925098496,positive,1.0000,NaN,NaN,American,NaN,jlhalldc,NaN,0,Thank you. “@AmericanAir: @jlhalldc Customer R...,NaN,2015-02-22 12:04:51 -0800,"Washington, DC",Eastern Time (US & Canada)
8691,568014845062332416,negative,1.0000,Late Flight,1.0000,Delta,NaN,rjburnsva,NaN,0,@JetBlue #1408 IAD to JFK still hasn't boarded...,NaN,2015-02-18 03:51:06 -0800,NaN,Eastern Time (US & Canada)
12252,570250940290342914,negative,0.6866,Lost Luggage,0.3520,American,NaN,Kukirani,NaN,0,@AmericanAir hi we have lost and found solutio...,NaN,2015-02-24 07:56:33 -0800,Worldwide,NaN
13866,569699154965827584,negative,0.6765,Flight Booking Problems,0.6765,American,NaN,kathyjazztx,NaN,0,@AmericanAir and it gets better...other passen...,NaN,2015-02-22 19:23:57 -0800,NaN,NaN
5308,569206183283843072,negative,0.6500,Can't Tell,0.3609,Southwest,NaN,Tweeter_mom_,NaN,0,@SouthwestAir My husband is responding to him ...,NaN,2015-02-21 10:45:04 -0800,Florida,Eastern Time (US & Canada)
8648,568083588308197376,negative,1.0000,Late Flight,1.0000,Delta,NaN,duendecillita,NaN,0,@JetBlue you are missing the point. The flight...,NaN,2015-02-18 08:24:16 -0800,NYC,Eastern Time (US & Canada)
5379,569158165176164353,negative,1.0000,Late Flight,1.0000,Southwest,NaN,bsjuts,NaN,0,@SouthwestAir where are our pilots? Plane is h...,NaN,2015-02-21 07:34:15 -0800,Nebraska,Central Time (US & Canada)
3149,568637541513089024,negative,0.9220,Customer Service Issue,0.4513,United,negative,Mosborne13,Cancelled Flight\nCustomer Service Issue,0,@united rebooked 24 hours after original fligh...,NaN,2015-02-19 21:05:29 -0800,NaN,Central Time (US & Canada)
433,568083179023650818,negative,1.0000,Bad Flight,0.3556,Virgin America,NaN,iamWalkerR,NaN,0,@VirginAmerica kinda sucked my earphone jack d...,NaN,2015-02-18 08:22:39 -0800,"New York, NY",Central Time (US & Canada)


In [23]:
airlines_sentiment_df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

Selecting only airline_sentiment and text filed.

In [24]:
airlines_sentiment_df = airlines_sentiment_df[['airline_sentiment', 'text']]

In [25]:
airlines_sentiment_df.head()

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


**This is my  data set $\mathcal{D} = \{({x_i}, y_i)\}_{i=1}^{N=5574}$ $x_i$ is text field and $y_i$ is airlines_sentiment(positive or negative)**. Using using this I will train(learn parameters $\theta$ of a models(Naive bayes, Discriminant anlaysis based etc.)) and use trained model to classify new text as positive or negative.


In [26]:
airlines_sentiment_df.dtypes

airline_sentiment    object
text                 object
dtype: object

In [27]:
airlines_sentiment_df.isna().sum()

airline_sentiment    0
text                 0
dtype: int64

Dataset doesn't contain any nan and  looks clean.

## Converting text to vectors(feature).

# Feature engineering

**Some [lemmatization](https://en.wikipedia.org/wiki/Lemmatisation)**

In [28]:
airlines_sentiment_df.head()

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [29]:
import gensim
from gensim.utils import lemmatize

In [30]:
try :lemmatize("Hi Machine learning")
except :pass
print(lemmatize("Hi Machine learning"))

[b'machine/NN', b'learn/VB']


In [31]:
lemmatized_out =[wd.decode('utf-8').split('/')[0] for wd in lemmatize("Hi Machine learning")]
print(lemmatized_out)

['machine', 'learn']


In [32]:
preprocessed_messages= airlines_sentiment_df.text.apply(lambda text: [wd.decode('utf-8').split('/')[0] for wd in lemmatize(text)])

After lemmatization:

In [33]:
preprocessed_messages.head()

1    [virginamerica, ve, add, commercial, experienc...
3    [virginamerica, really, aggressive, blast, obn...
4             [virginamerica, really, big, bad, thing]
5    [virginamerica, seriously, pay, flight, seat, ...
6    [virginamerica, yes, nearly, time, fly, vx, ea...
Name: text, dtype: object

In [34]:
messages_labels = airlines_sentiment_df.airline_sentiment

# Partitioning the data into training and test set

In [35]:
from sklearn.model_selection import train_test_split

training_messages, test_messages, training_labels, test_labels = train_test_split(preprocessed_messages,
                                                                                 messages_labels,
                                                                                test_size= .2,random_state=0,
                                                                                 stratify= messages_labels)
print('Shape of training X {} and train Y {}'.format(training_messages.shape, training_labels.shape))
print('Shape of test X {} and test Y {}'.format(test_messages.shape, test_labels.shape))

Shape of training X (9232,) and train Y (9232,)
Shape of test X (2309,) and test Y (2309,)


In [36]:
unique_word = set()

for message in training_messages:
        unique_word.update(message)

In [37]:
# how many words in vocabulary
num_unique_words = len(unique_word)
num_unique_words

8192

Some **feature engineering**. 

In [38]:
#first  build a default dictionary to assign each word a unique location in the feature vector
from collections import defaultdict, Counter
word_to_index_dict = defaultdict(int)
for index , word in enumerate(unique_word):
    word_to_index_dict[word] = index

In [39]:
# create a reverse dictionary  for mapping index to word. It will help in debugging etc.
index_to_word_dict = { value:key  for key, value in word_to_index_dict.items()}

In [40]:
num_messages = training_messages.shape
num_messages

(9232,)

create a numpy integer matrix of **num_messages X num_unique_words**, initialized with zero

In [41]:
training_X = np.zeros((len(training_messages), len(unique_word)), dtype=int)
print(training_X.shape)
test_X = np.zeros((len(test_messages), len(unique_word)), dtype=int)

(9232, 8192)


In [42]:
def build_feature(text, word_to_index_dict):
    feature = np.zeros((len(word_to_index_dict),), dtype=int)
    word_freq =  Counter(text)
    # setting the word count in text_no row of text_features
    for word, freq in word_freq.items():
        if word in word_to_index_dict:
            index_of_word = word_to_index_dict[word]
            feature[index_of_word] = freq
    return feature        
for text_no, text in enumerate(training_messages):
    training_X[text_no] = build_feature(text, word_to_index_dict)
    
for text_no, text in enumerate(test_messages):
    test_X[text_no] = build_feature(text, word_to_index_dict)

## Re checking the above results

In [43]:
text_no =7
message_word_count = Counter(training_messages.iloc[text_no])
print(message_word_count)
print('##Encoding for text no {} in feature vector is ##'.format(text_no))
for i, count in enumerate(training_X[text_no]):
    if count >0:
        print(index_to_word_dict[i], count)


Counter({'ewr': 2, 'americanair': 1, 'itinerary': 1, 'be': 1, 'dalla': 1, 'la': 1, 'cancel': 1, 'flightle': 1, 'flight': 1, 'have': 1, 'money': 1, 'find': 1, 'way': 1, 'get': 1, 'there': 1})
##Encoding for text no 7 in feature vector is ##
americanair 1
la 1
cancel 1
way 1
be 1
flight 1
dalla 1
there 1
flightle 1
money 1
find 1
have 1
itinerary 1
ewr 2
get 1


# Converting positive and negative label to 1 and 0  respectively

In [44]:
training_labels.tail(7)# can check from head too

11874    negative
1986     negative
4363     negative
4215     positive
7737     positive
8320     negative
10617    negative
Name: airline_sentiment, dtype: object

In [45]:
training_y = (training_labels.values == 'positive').astype(int)
test_y = (test_labels.values == 'positive').astype(int)

In [46]:
# summing up per feature count
training_X_positive = training_X[training_y ==1]
print(training_X_positive.shape)
per_feature_count =np.sum(training_X_positive, axis = 0)
print(per_feature_count.shape)

np.count_nonzero(per_feature_count)
parameters_w_positive = per_feature_count/(np.sum(per_feature_count))

parameters_w_positive

(1890, 8192)
(8192,)


array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00011492])

# Estimating parameters for negative

In [47]:
training_X_negative = training_X[training_y ==0]
print(training_X_negative.shape)
per_feature_count =np.sum(training_X_negative, axis = 0)
print(per_feature_count.shape)

np.count_nonzero(per_feature_count)
parameters_w_negative = per_feature_count/(np.sum(per_feature_count))


print(parameters_w_negative)

(7342, 8192)
(8192,)
[8.99391786e-05 5.62119866e-05 1.12423973e-05 ... 1.12423973e-05
 1.12423973e-05 1.23666371e-04]


# Zero probability issue
As some of the probability can be zero. It will create problem when trying to estimate probability of a new document in test set if that was not in training set. 

**One way to handle this situtation to add a fake 1 count of the word in each class called Laplace law of succession or add one smoothing**.

# Building a class for our multinomial naive bayes and estimate its new parameters based on  Laplace law of succession.

In [48]:
class multinomial_naive_bayes(object):
    
    NEGATIVE_CLASS, POSITIVE_CLASS = range(2)
    
    def __init__(self, class_prior=(.5, .5), fit_prior=True):
        
        self._class_prior= class_prior
        self._fit_prior = fit_prior
        
        
    def fit(self, training_X, training_y):
        self.training_X_positive = training_X[training_y ==1]
        self.training_X_positive = np.append(self.training_X_positive, np.ones(shape=(1, self.training_X_positive.shape[1])), axis=0)
        self.per_feature_count_positive =np.sum(self.training_X_positive, axis = 0)
        self._parameters_w_positive = self.per_feature_count_positive/(np.sum(self.per_feature_count_positive))
        
        self.training_X_negative = training_X[training_y ==0]
        self.training_X_negative = np.append(self.training_X_negative, np.ones(shape=(1, self.training_X_negative.shape[1])), axis=0)
        self.per_feature_count_negative =np.sum(self.training_X_negative, axis = 0)
        self._parameters_w_negative = self.per_feature_count_negative/(np.sum(self.per_feature_count_negative))
        
        if self._fit_prior:
            self._class_prior = self._calculate_prior(len(self.training_X_positive), len(self.training_X_negative))
         
    def _calculate_prior(self, n_positive, n_negative):    
        positive_prior = n_positive/(n_positive + n_negative)
        negative_prior = 1- positive_prior
        return negative_prior,positive_prior
        #raise NotImplementedError("implement this!")
        
        
    def _calculate_score(self, parameters,test_text, class_prior):
            return np.sum(np.log(np.power(parameters,test_text))) + class_prior            
        
    def predict(self, test_X, test_y):
        self.positive_score = np.zeros_like(test_y,dtype=float)
        self.negative_score = np.zeros_like(test_y,dtype=float)

        for idx, test_text in enumerate(test_X):# this will fetch row by row, encoded test messages
            self.positive_score[idx] = self._calculate_score(self._parameters_w_positive,
                                                        test_text,
                                                        np.log(self._class_prior[multinomial_naive_bayes.POSITIVE_CLASS]))
            self.negative_score[idx] = self._calculate_score(self._parameters_w_negative,
                                                        test_text, np.log(self._class_prior[multinomial_naive_bayes.NEGATIVE_CLASS]))

       # predict the label positive(1) or negative(0)
        positive_or_negative = (self.positive_score >= self.negative_score).astype(int)
        accuracy= np.sum(positive_or_negative == test_y) / len(positive_or_negative)
        return accuracy    

In [49]:
clf = multinomial_naive_bayes()

In [50]:
clf.fit(training_X, training_y)

In [51]:
clf.predict(test_X, test_y)

0.9060199220441749

# Implementing using MultinomialNB classifer  from sklearn and report accuracy in test set

In [52]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(training_X, training_y)
ypred= classifier.predict(test_X)
classifier.score(test_X, test_y)

0.9060199220441749